In [2]:
import numpy as np
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pyspark.sql.functions as F
from pyspark.sql.functions import when, udf, col, regexp_replace
from pyspark.sql.types import DoubleType,IntegerType, StringType 

In [3]:
sc = SparkContext( 'local' )  
sqlCtx = SQLContext( sc )

In [6]:
df = sqlCtx.read.csv('data/crime_in_Seoul.csv',encoding='euc-kr',
                     header=True, inferSchema=True)
df.show()

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|    1,355|    1,170|
|  종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|    1,278|    1,070|
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
|  혜화서|        3|        2|        5|        4|       96|       63|    1,114|      424|    1,015|      861|
|  용산서|        5|        5|       14|       14|      194|      173|    1,557|      587|    2,050|    1,704|
|  성북서|        2|        2|        2|        1|       86|       

In [10]:
df.select('관서명').rdd.map(lambda x: x['관서명']).collect()

['중부서',
 '종로서',
 '남대문서',
 '서대문서',
 '혜화서',
 '용산서',
 '성북서',
 '동대문서',
 '마포서',
 '영등포서',
 '성동서',
 '동작서',
 '광진서',
 '서부서',
 '강북서',
 '금천서',
 '중랑서',
 '강남서',
 '관악서',
 '강서서',
 '강동서',
 '종암서',
 '구로서',
 '서초서',
 '양천서',
 '송파서',
 '노원서',
 '방배서',
 '은평서',
 '도봉서',
 '수서서']

In [14]:
# colume 객체의 멤버함수..
# df['관서명'].cast()
# df['관서명'].isin()
# df['관서명'].like()
# df['관서명'].rlike()

In [13]:
# where ( 문자열 or 컬럼객체 )
df.where(df['관서명'].isin('중부서','종로서')).show()

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|    1,355|    1,170|
|종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|    1,278|    1,070|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [15]:
df.where(df['관서명'].like('%북%')).show()

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|성북서|        2|        2|        2|        1|       86|       71|      953|      409|    1,194|    1,015|
|강북서|        7|        8|       14|       13|      153|      126|    1,434|      618|    2,649|    2,348|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [17]:
df.where(df['관서명'].rlike('문')).show()   # rlike => 정규식 패턴

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
|동대문서|        5|        5|       13|       13|      173|      146|    1,981|      814|    2,548|    2,227|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [18]:
df.where(df['관서명'].rlike('^서')).show()  

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
|  서부서|        2|        2|        2|        1|       70|       59|      819|      293|    1,192|    1,038|
|  서초서|        7|        4|        8|        5|      334|      193|    1,982|      905|    1,852|    1,607|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [19]:
df.where(df['관서명'].rlike('서$')).show()  

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|    1,355|    1,170|
|  종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|    1,278|    1,070|
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
|  혜화서|        3|        2|        5|        4|       96|       63|    1,114|      424|    1,015|      861|
|  용산서|        5|        5|       14|       14|      194|      173|    1,557|      587|    2,050|    1,704|
|  성북서|        2|        2|        2|        1|       86|       

In [21]:
df.where(df['관서명'].rlike('^마|동|강')).show()  

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|동대문서|        5|        5|       13|       13|      173|      146|    1,981|      814|    2,548|    2,227|
|  마포서|        8|        8|       14|       10|      294|      247|    2,555|      813|    2,983|    2,519|
|  성동서|        4|        4|        9|        8|      126|      119|    1,607|      597|    1,612|    1,395|
|  동작서|        5|        5|        9|        5|      285|      139|    1,865|      661|    1,910|    1,587|
|  강북서|        7|        8|       14|       13|      153|      126|    1,434|      618|    2,649|    2,348|
|  강남서|        3|        3|       15|       12|      300|      225|    2,411|      984|    2,465|    2,146|
|  강서서|        7|        8|       13|       13|      262|      

In [23]:
df.where(df['관서명'].rlike('강[북남]')).show()  

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|강북서|        7|        8|       14|       13|      153|      126|    1,434|      618|    2,649|    2,348|
|강남서|        3|        3|       15|       12|      300|      225|    2,411|      984|    2,465|    2,146|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+

